In [1]:
from pyspark.sql.functions import max, min, avg, count, round

In [5]:
catalog = 'citibike_dev'

In [6]:
# catalog = dbutils.widgets.get("catalog")

In [7]:

df = spark.read.table(f"{catalog}.02_silver.jc_citibike")

In [8]:
df.first()

Row(ride_id='29DAF43DD84B4B7A', trip_start_date=datetime.date(2025, 3, 20), started_at=datetime.datetime(2025, 3, 20, 14, 58, 31, 217000), ended_at=datetime.datetime(2025, 3, 20, 15, 0, 46, 466000), start_station_name='6 St & Grand St', end_station_name='Mama Johnson Field - 4 St & Jackson St', trip_duration_mins=2.25, metadata={'pipeline_id': 'placeholder', 'run_id': 'placeholder', 'task_id': 'placeholder', 'processed_timestamp': 'placeholder'})

In [9]:

df = df.groupBy("trip_start_date").agg(
    round(max("trip_duration_mins"),2).alias("max_trip_duration_mins"),
    round(min("trip_duration_mins"),2).alias("min_trip_duration_mins"),
    round(avg("trip_duration_mins"),2).alias("avg_trip_duration_mins"),
    count("ride_id").alias("total_trips")
)

In [10]:
df.show()

+---------------+----------------------+----------------------+----------------------+-----------+
|trip_start_date|max_trip_duration_mins|min_trip_duration_mins|avg_trip_duration_mins|total_trips|
+---------------+----------------------+----------------------+----------------------+-----------+
|     2025-03-23|                559.77|                  1.03|                 10.17|       1875|
|     2025-03-16|                202.63|                  1.07|                  9.43|       1928|
|     2025-03-03|                 138.7|                  1.02|                  6.53|       2030|
|     2025-03-08|                  91.6|                  1.02|                  7.35|       2008|
|     2025-02-28|                 25.25|                   2.8|                  11.4|          4|
|     2025-03-12|                951.33|                  1.02|                  7.46|       2687|
|     2025-03-07|               1499.93|                  1.03|                  8.21|       2338|
|     2025

In [11]:
df.write.\
    mode("overwrite").\
    option("overwriteSchema", "true").\
    saveAsTable(f"{catalog}.03_gold.daily_ride_summary")